# Egenverdier og evenvektorer (MIP 10.8)

In [1]:
import sympy as sp
import numpy as np

## Python kode

In [2]:
def normer_forste_element(vektor):
    """
    Normaliserer en vektor ved å dele alle elementer på det første ikke-null elementet.
    
    Parametere:
        vektor (np.ndarray): En 1D Numpy-array som skal normaliseres.
    
    Returnerer:
        np.ndarray: En normalisert vektor der det første ikke-null elementet er 1.
    """
    if np.allclose(vektor, 0):
        return vektor
    # Finn indeksen til det første elementet i vektoren som ikke er null
    forste_ikke_null_indeks = np.argmax(vektor != 0)
    
    # Hent verdien til det første ikke-null elementet
    forste_ikke_null_element = vektor[forste_ikke_null_indeks]
    
    # Returner den normaliserte vektoren
    return vektor / forste_ikke_null_element

def gauss_jordan(matrise, epsilon=1e-8):
    """
    Utfører Gauss-Jordan eliminasjon på en gitt matrise.
    
    Parametere:
        matrise (np.ndarray): En 2D Numpy-array som representerer matrisen.
        epsilon (float): En liten verdi for å håndtere numerisk presisjon.
    
    Returnerer:
        np.ndarray: En rekkeredusert matrise i redusert trappeform.
    """
    # Sett svært små verdier til null for å unngå numeriske feil
    matrise[np.abs(matrise) < epsilon] = 0
    
    # Hvis matrisen kun består av nuller, returner den uendret
    if np.allclose(matrise, 0):
        return matrise
    
    # Hvis matrisen har én rad, normaliser den første radens første ikke-null element
    elif len(matrise) == 1:
        matrise[0] = normer_forste_element(matrise[0])
        return matrise
    
    # Beregn radenes summer for å normalisere matrisen (gjør tallene mer håndterbare)
    rad_maksimummer = np.max(np.abs(matrise), axis=1)
    rad_maksimummer[rad_maksimummer <= epsilon] = 1
    matrise = matrise / rad_maksimummer[:, None]  # Normaliser hver rad
    # Finn kolonner som inneholder ikke-null elementer
    ikke_null_kolonner = np.any(matrise != 0, axis=0)
    
    # Finn indeksen til den første kolonnen med ikke-null elementer
    forste_ikke_null_kolonne = np.argmax(np.abs(ikke_null_kolonner))
    # Finn indeksen til raden med største verdi i den valgte kolonnen (pivot rad)
    pivot_rad_indeks = np.argmax(matrise[:, forste_ikke_null_kolonne])
    
    # Normaliser pivot-raden
    pivot_rad = normer_forste_element(matrise[pivot_rad_indeks])
    
    # Bytt plass på pivot-raden og den første raden
    matrise[pivot_rad_indeks] = matrise[0]
    matrise[0] = pivot_rad
    # Utfør eliminering for å gjøre alle elementene under pivoten null
    if matrise[0, forste_ikke_null_kolonne] > epsilon:
        matrise[1:] -= (matrise[1:, 0] / matrise[0, forste_ikke_null_kolonne])[:, None] * matrise[0]

    # Kall Gauss-Jordan rekursivt på den nedre delmatrisen
    matrise[1:, 1:] = gauss_jordan(matrise[1:, 1:])
    
    # Gjør den første raden null over pivot-posisjonene til de øvrige radene.
    for rad in matrise[1:]:
        if np.any(rad != 0):  # Hvis raden ikke er null
            # Finn indeksen til det første ikke-null elementet i raden
            forste_ikke_null_kolonne = np.argmax(rad != 0)
            
            # Trekk fra et multiplum av denne raden for å gjøre elementet over pivot null
            matrise[0] -= (matrise[0, forste_ikke_null_kolonne] / rad[forste_ikke_null_kolonne]) * rad
    
    # Returner den resulterende matrisen
    return matrise


In [3]:
def pivot_posisjoner(matrise):
    """
    Finner pivotposisjonene i en rekkeredusert matrise.
    """
    pivot_sett = set()
    pivot_rader = []
    pivot_kolonner = []
    for rad_indeks, rad in enumerate(matrise):
        if np.any(rad != 0):
            første_ikke_null_kolonne = np.argmax(rad != 0)
            pivot_rader.append(rad_indeks)
            pivot_kolonner.append(første_ikke_null_kolonne)
    return pivot_rader, pivot_kolonner

def frie_parametre(matrise):
    """
    Finner bundne og frie parametere i en rekkeredusert matrise.
    """
    _, pivot_kolonner = pivot_posisjoner(matrise)
    alle_kolonner = set(range(matrise.shape[1]))
    return sorted(alle_kolonner.difference(pivot_kolonner))

def null_rom(matrise):
    """
    Finner en basis for nullrommet til en matrise.
    """
    nullrom_basis = []
    redusert_matrise = gauss_jordan(matrise)
    pivot_rader, pivot_kolonner = pivot_posisjoner(redusert_matrise)
    frie = frie_parametre(redusert_matrise)
    
    for fri in frie:
        vektor = np.zeros((matrise.shape[1], 1), dtype=matrise.dtype)
        vektor[fri] = 1
        for rad, kolonne in zip(pivot_rader, pivot_kolonner):
            vektor[kolonne] = -np.sum((redusert_matrise @ vektor)[rad])
        nullrom_basis.append(vektor)
    
    return nullrom_basis

def partikulaer_losning(koeffisientmatrise, høyreside=None):
    """
    Finner en partikulær løsning til ligningssystemet Ax = b.
    """
    if høyreside is None:
        høyreside = np.zeros(koeffisientmatrise.shape[1])
    if len(høyreside.shape) == 1:
        høyreside = høyreside[:, None]
    
    utvidet_matrise = gauss_jordan(np.hstack([koeffisientmatrise, høyreside]))
    radindekser, kolonneindekser = pivot_posisjoner(utvidet_matrise[:, :-1])
    løsning = np.zeros((koeffisientmatrise.shape[1], 1))
    redusert_høyreside = utvidet_matrise[:, -1]
    
    for rad, kolonne in zip(radindekser, kolonneindekser):
        løsning[kolonne] = redusert_høyreside[rad]

    assert np.allclose(koeffisientmatrise @ løsning[None, :], høyreside), "Det finnes ingen løsning til det lineære ligningssystemet"
    return løsning


In [4]:
def finn_egenvektorer_og_egenverdier(A):
    assert A.shape[0] == A.shape[1], "matrisen A skal være kvadratisk"

    t = sp.symbols('t')  # Definerer symbolet t, som brukes i det karakteristiske polynomet
    B = A - t * sp.eye(A.shape[0])  # Lager matrisen B = A - t*I, hvor I er identitetsmatrisen
    karakteristisk_polynom = B.det()  # Finner determinant av B, som gir det karakteristiske polynomet
    
    egenverdier = sp.solve(karakteristisk_polynom)  # Løser det karakteristiske polynomet for å finne egenverdiene

    # Lager en liste med tupler av evenverdier, multiplisiteter og egenvektorer
    res = []
    # for ev in sorted(egenverdier, key=lambda x: -np.abs(x)):
    for ev in egenverdier:
        ev = complex(ev)
        if ev.real == ev or np.iscomplexobj(A):
            if ev.real == ev:
                ev = ev.real
            egenvektorer = null_rom((A - ev * np.eye(A.shape[0])))
            res.append((ev, len(egenvektorer), egenvektorer))
    
    return res  # Returnerer en liste med tupler av evenverdier, multiplisiteter og egenvektorer

In [5]:
def complex_to_string(c):
    if c.imag == 0:
        return '{0.real:.2f}'.format(c)
    if c.real == 0:
        return '{0.imag:.2f}j'.format(c)
    return '{0.real:.2f} + {0.imag:.2f}j'.format(c)


In [6]:
def complex_to_string(c, precision=7):
    if c.imag == 0:
        return '{0.real:.{0}f}'.format(precision, c)
    if c.real == 0:
        return '{0.imag:.{0}f}j'.format(precision, c)
    return '{0.real:.{0}f} + {0.imag:.{0}f}j'.format(precision, c)

In [7]:
def complex_to_string(c, precision=7):
    if np.all(c.imag == 0):
        return np.array2string(c.real, precision=precision)
    else:
        return np.array2string(c, precision=precision)

In [8]:
def skriv_ut_egenvektorer_og_multiplikasjon_med_matrise(A, egenverdier_og_egenvektorer, presisjon=3):
    print('Alle vektorer her skal leses som kolonnevektorer\n')
    for key, _, val in sorted(
        egenverdier_og_egenvektorer, 
        key = lambda x: (complex(x[0]).real)**2 + (complex(x[0]).imag)**2):
        print('egenverdi:     ', 
              key)
        for v in val:
            print('egenvektor:    ', complex_to_string(np.array(v, dtype='complex128').ravel(), precision=presisjon))
            print('A @ evenvektor:', complex_to_string(np.array(A @ v, dtype='complex128').ravel(), precision=presisjon))
        print()

In [9]:
def skriv_ut_numpy_egenvektorer_og_multiplikasjon_med_matrise(A, presisjon=3):
    print('Alle vektorer her skal leses som kolonnevektorer\n')
    eigenverdier, eigenvektorer = np.linalg.eig(A)
    for key, val in sorted(zip(eigenverdier, eigenvektorer.T),
                           key = lambda x: (complex(x[0]).real)**2 + (complex(x[0]).imag)**2):
        v = val[:, None]
        print('egenverdi:     ', 
              key)
        print('egenvektor:    ', complex_to_string(np.array(v, dtype='complex128').ravel(), precision=presisjon))
        print('A @ evenvektor:', complex_to_string(np.array(A @ v, dtype='complex128').ravel(), precision=presisjon))
        print()

## Eksempel

In [10]:
A = np.array([
    [0, .5, .5],
    [1, 0, 0],
    [0, .5, .5]
])

In [11]:
skriv_ut_egenvektorer_og_multiplikasjon_med_matrise(A, finn_egenvektorer_og_egenverdier(A))

Alle vektorer her skal leses som kolonnevektorer

egenverdi:      0.0
egenvektor:     [-0. -1.  1.]
A @ evenvektor: [0. 0. 0.]

egenverdi:      -0.5
egenvektor:     [ 1. -2.  1.]
A @ evenvektor: [-0.5  1.  -0.5]

egenverdi:      1.0
egenvektor:     [1. 1. 1.]
A @ evenvektor: [1. 1. 1.]



Pakken sympy kan regne ut egenverdier og egenvektorer for oss. Uheldigvis regner den ikke helt presis!

In [12]:
skriv_ut_egenvektorer_og_multiplikasjon_med_matrise(A, sp.Matrix(A).eigenvects())

Alle vektorer her skal leses som kolonnevektorer

egenverdi:      -7.10753320192609e-65
egenvektor:     [ 4.162e-65 -8.165e-01  8.165e-01]
A @ evenvektor: [0.000e+00 4.162e-65 0.000e+00]

egenverdi:      -0.500000000000000
egenvektor:     [ 0.408 -0.816  0.408]
A @ evenvektor: [-0.204  0.408 -0.204]

egenverdi:      1.00000000000000
egenvektor:     [0.577 0.577 0.577]
A @ evenvektor: [0.577 0.577 0.577]



Også numpy kan regne ut egenverdier og egenvektorer. Uheldigvis regner den heller ikke helt presis.

In [13]:
skriv_ut_numpy_egenvektorer_og_multiplikasjon_med_matrise(A)

Alle vektorer her skal leses som kolonnevektorer

egenverdi:      2.6086068435760837e-17
egenvektor:     [-1.110e-16 -7.071e-01  7.071e-01]
A @ evenvektor: [ 3.331e-16 -1.110e-16  3.331e-16]

egenverdi:      -0.5000000000000006
egenvektor:     [ 0.408 -0.816  0.408]
A @ evenvektor: [-0.204  0.408 -0.204]

egenverdi:      0.9999999999999998
egenvektor:     [-0.577 -0.577 -0.577]
A @ evenvektor: [-0.577 -0.577 -0.577]



## Hva skjete her?

For 
$$
A = \left[\begin{array}{ccc}
0 & 1/2 & 1/2 \\ 1 & 0 & 0 \\ 0 & 1/2 & 1/2
\end{array}\right]$$
er
$$
A \cdot
\left[\begin{array}{c}
1 \\ -2 \\ 1
\end{array}\right]
= \left[\begin{array}{ccc}
0 & 1/2 & 1/2 \\ 1 & 0 & 0 \\ 0 & 1/2 & 1/2
\end{array}\right]
\left[\begin{array}{c}
1 \\ -2 \\ 1
\end{array}\right]
=
\left[\begin{array}{c}
-1/2 \\ 1 \\ -1/2
\end{array}\right]
= 
-\frac 12 
\left[\begin{array}{c}
1 \\ -2 \\ 1
\end{array}\right]
$$


Vi sier at 
$\left[\begin{array}{c}
1 \\ -2 \\ 1
\end{array}\right]$
er en **egenvektor** til 
$
A = \left[\begin{array}{ccc}
0 & 1/2 & 1/2 \\ 1 & 0 & 0 \\ 0 & 1/2 & 1/2
\end{array}\right]$
med **egenverdi** $-\frac 12$

Tilsvarende er
$\left[\begin{array}{c}
1 \\ -1 \\ 0
\end{array}\right]$
en **egenvektor** til 
$
A = \left[\begin{array}{ccc}
0 & 1/2 & 1/2 \\ 1 & 0 & 0 \\ 0 & 1/2 & 1/2
\end{array}\right]$
med **egenverdi** $0$

og
$\left[\begin{array}{c}
1 \\ 1 \\ 1
\end{array}\right]$
er en **egenvektor** til 
$
A = \left[\begin{array}{ccc}
0 & 1/2 & 1/2 \\ 1 & 0 & 0 \\ 0 & 1/2 & 1/2
\end{array}\right]$
med **egenverdi** $1$

### Oppgave:
Sjekk at 
$
\left[\begin{array}{ccc}
0 & 1/2 & 1/2 \\ 1 & 0 & 0 \\ 0 & 1/2 & 1/2
\end{array}\right] \cdot 
\left[\begin{array}{c}
1 \\ 1 \\ 1
\end{array}\right]
= 
1\cdot
\left[\begin{array}{c}
1 \\ 1 \\ 1
\end{array}\right]
$
og at
$
\left[\begin{array}{ccc}
0 & 1/2 & 1/2 \\ 1 & 0 & 0 \\ 0 & 1/2 & 1/2
\end{array}\right] \cdot 
\left[\begin{array}{c}
1 \\ -1 \\ 0
\end{array}\right]
= 
0\cdot
\left[\begin{array}{c}
1 \\ -1 \\ 0
\end{array}\right]
$

## Metode for å finne egenverdier og egenvektorer

Hvis $A \cdot \vec v = t \vec v$, da er $\vec 0 = A \cdot \vec v - t \vec v = A \cdot \vec v - t I \cdot \vec v = (A - tI)\vec v$. Hvis $\vec v \ne \vec 0$ må vi ha $\det(A - tI) = 0$.

Omvendt, hvis $\det(A - tI) = 0$, da vet vi at det finnes in $\vec v \ne \vec 0$ med $(A - tI) \cdot \vec v = \vec 0$.

Vi ser på $\det(A - tI)$ som en funksjon i den ubestemte $t$, og finner alle nullpunkter for denne funksjonen. Her kan vi for eksempel anvende metoden til Newton. 
Hvis $A$ er en $2 \times 2$ eller $3 \times 3$ matrise da er $\det(A - tI)$ et polynom av grad henholdsvis 2 eller 3. Da kan vi, med litt jobbing og litt hell,
finne nullpunkter for hånd.

## Eksempel

In [14]:
A = np.array([
    [2, 1, 1],
    [2, 3, 4],
    [1, -1, 2]
])

In [15]:
skriv_ut_egenvektorer_og_multiplikasjon_med_matrise(A, finn_egenvektorer_og_egenverdier(A))

Alle vektorer her skal leses som kolonnevektorer

egenverdi:      3.0
egenvektor:     [-2. -3.  1.]
A @ evenvektor: [-6. -9.  3.]



#### Resten av dette eksempelet er bare for spesielt interesserte

In [16]:
t = sp.symbols('t')  # Definerer symbolet t, som brukes i det karakteristiske polynomet
B = A - t * sp.eye(A.shape[0])  # Lager matrisen B = A - t*I, hvor I er identitetsmatrisen
karakteristisk_polynom = B.det()  # Finner determinant av B, som gir det karakteristiske polynomet

egenverdier = sp.solve(karakteristisk_polynom)  # Løser det karakteristiske polynomet for å finne egenverdiene

In [17]:
egenverdier

[3, 2 - I, 2 + I]

In [18]:
B

Matrix([
[2 - t,     1,     1],
[    2, 3 - t,     4],
[    1,    -1, 2 - t]])

In [19]:
karakteristisk_polynom

-t**3 + 7*t**2 - 17*t + 15

In [20]:
sp.simplify(karakteristisk_polynom / (t - 3))

-t**2 + 4*t - 5

Kan du forklare hvorfor vi kun finner en egenvektor?

In [21]:
skriv_ut_egenvektorer_og_multiplikasjon_med_matrise(A, finn_egenvektorer_og_egenverdier(A.astype('complex128')))

Alle vektorer her skal leses som kolonnevektorer

egenverdi:      (2-1j)
egenvektor:     [-1.-1.11e-16j -1.+1.00e+00j  1.+0.00e+00j]
A @ evenvektor: [-2.+1.j -1.+3.j  2.-1.j]

egenverdi:      (2+1j)
egenvektor:     [-1.+1.11e-16j -1.-1.00e+00j  1.+0.00e+00j]
A @ evenvektor: [-2.-1.j -1.-3.j  2.+1.j]

egenverdi:      3.0
egenvektor:     [-2. -3.  1.]
A @ evenvektor: [-6. -9.  3.]



Hva skjete nå?

## Eksempel

In [22]:
M = np.array([
    [-4, -2, -2],
    [-4, -6, -8],
    [2, 2, 4]
])

In [23]:
skriv_ut_egenvektorer_og_multiplikasjon_med_matrise(M, finn_egenvektorer_og_egenverdier(M))

Alle vektorer her skal leses som kolonnevektorer

egenverdi:      -2.0
egenvektor:     [-1.  1. -0.]
A @ evenvektor: [ 2. -2.  0.]

egenverdi:      2.0
egenvektor:     [-0. -1.  1.]
A @ evenvektor: [ 0. -2.  2.]

egenverdi:      -6.0
egenvektor:     [-2. -3.  1.]
A @ evenvektor: [12. 18. -6.]



In [24]:
t = sp.symbols('t')  # Definerer symbolet t, som brukes i det karakteristiske polynomet
B = M - t * sp.eye(M.shape[0])  # Lager matrisen B = A - t*I, hvor I er identitetsmatrisen
karakteristisk_polynom = B.det()  # Finner determinant av B, som gir det karakteristiske polynomet

egenverdier = sp.solve(karakteristisk_polynom)  # Løser det karakteristiske polynomet for å finne egenverdiene
egenverdier

[-6, -2, 2]

In [25]:
B

Matrix([
[-t - 4,     -2,    -2],
[    -4, -t - 6,    -8],
[     2,      2, 4 - t]])

In [26]:
karakteristisk_polynom

-t**3 - 6*t**2 + 4*t + 24

## Eksempel

In [27]:
A = np.array([
    [7, 0, -4],
    [0, 5, 0],
    [5, 0, -2]
])

In [28]:
skriv_ut_egenvektorer_og_multiplikasjon_med_matrise(A, finn_egenvektorer_og_egenverdier(A))

Alle vektorer her skal leses som kolonnevektorer

egenverdi:      2.0
egenvektor:     [ 0.8 -0.   1. ]
A @ evenvektor: [1.6 0.  2. ]

egenverdi:      3.0
egenvektor:     [ 1. -0.  1.]
A @ evenvektor: [3. 0. 3.]

egenverdi:      5.0
egenvektor:     [-0.  1. -0.]
A @ evenvektor: [0. 5. 0.]



## Eksempel

In [29]:
A = np.array([
    [1, 1, -3],
    [2, 0, 6],
    [1, -1, 5]
])

In [30]:
skriv_ut_egenvektorer_og_multiplikasjon_med_matrise(A, finn_egenvektorer_og_egenverdier(A.astype('complex128')))

Alle vektorer her skal leses som kolonnevektorer

egenverdi:      2.0
egenvektor:     [1. 1. 0.]
A @ evenvektor: [2. 2. 0.]
egenvektor:     [-3.  0.  1.]
A @ evenvektor: [-6.  0.  2.]



## Eksempel

In [31]:
A = np.array([
    [1, -2, 3],
    [2, 6, -6],
    [1, 2, -1]
])

In [32]:
skriv_ut_egenvektorer_og_multiplikasjon_med_matrise(A, finn_egenvektorer_og_egenverdier(A))

Alle vektorer her skal leses som kolonnevektorer

egenverdi:      2.0
egenvektor:     [-2.  1.  0.]
A @ evenvektor: [-4.  2.  0.]
egenvektor:     [3. 0. 1.]
A @ evenvektor: [6. 0. 2.]



## Eksempel

In [33]:
A = np.array([
    [0, 1, 0],
    [3, 0, 1],
    [2, 0, 0]
])

In [34]:
skriv_ut_egenvektorer_og_multiplikasjon_med_matrise(A, finn_egenvektorer_og_egenverdier(A))

Alle vektorer her skal leses som kolonnevektorer

egenverdi:      -1.0
egenvektor:     [-0.5  0.5  1. ]
A @ evenvektor: [ 0.5 -0.5 -1. ]

egenverdi:      2.0
egenvektor:     [1. 2. 1.]
A @ evenvektor: [2. 4. 2.]



Også numpy kan regne ut egenverdier og egenvektorer. Uheldigvis regner den heller ikke helt presis. Den gir alltid tre egenvektorer selv om det kan finnes færre!

In [35]:
skriv_ut_numpy_egenvektorer_og_multiplikasjon_med_matrise(A)

Alle vektorer her skal leses som kolonnevektorer

egenverdi:      -0.9999999797236733
egenvektor:     [-0.408  0.408  0.816]
A @ evenvektor: [ 0.408 -0.408 -0.816]

egenverdi:      -1.0000000202763286
egenvektor:     [ 0.408 -0.408 -0.816]
A @ evenvektor: [-0.408  0.408  0.816]

egenverdi:      2.0
egenvektor:     [0.408 0.816 0.408]
A @ evenvektor: [0.816 1.633 0.816]



## Eksempel

In [36]:
A = np.array([
    [3, 1, 1],
    [-4, -2, -5],
    [2, 2, 5]
])

In [37]:
skriv_ut_egenvektorer_og_multiplikasjon_med_matrise(A, finn_egenvektorer_og_egenverdier(A))

Alle vektorer her skal leses som kolonnevektorer

egenverdi:      1.0
egenvektor:     [ 1. -3.  1.]
A @ evenvektor: [ 1. -3.  1.]

egenverdi:      2.0
egenvektor:     [-1.  1. -0.]
A @ evenvektor: [-2.  2.  0.]

egenverdi:      3.0
egenvektor:     [-0. -1.  1.]
A @ evenvektor: [ 0. -3.  3.]



## Eksempel

In [38]:
A = np.array([
    [2, 1, 1],
    [0, 1, 0],
    [1, -1, 2]
])

In [39]:
skriv_ut_egenvektorer_og_multiplikasjon_med_matrise(A, finn_egenvektorer_og_egenverdier(A))

Alle vektorer her skal leses som kolonnevektorer

egenverdi:      1.0
egenvektor:     [-1. -0.  1.]
A @ evenvektor: [-1.  0.  1.]

egenverdi:      3.0
egenvektor:     [ 1. -0.  1.]
A @ evenvektor: [3. 0. 3.]



## Eksempel

In [40]:
A = np.array([
    [1, 0, 0],
    [0, 1, 0],
    [0, 0, 2]
])

In [41]:
skriv_ut_egenvektorer_og_multiplikasjon_med_matrise(A, finn_egenvektorer_og_egenverdier(A))

Alle vektorer her skal leses som kolonnevektorer

egenverdi:      1.0
egenvektor:     [ 1.  0. -0.]
A @ evenvektor: [1. 0. 0.]
egenvektor:     [ 0.  1. -0.]
A @ evenvektor: [0. 1. 0.]

egenverdi:      2.0
egenvektor:     [-0. -0.  1.]
A @ evenvektor: [0. 0. 2.]

